In [1]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.0/350.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

# import colab secrets to store login credentials
from google.colab import userdata

# datetime is necessary for caturday check
import datetime
import zoneinfo

In [3]:
# vision transformer documentation - https://huggingface.co/docs/transformers/v4.13.0/en/model_doc/vit
# guide to using vision transformer from DataCamp - https://www.youtube.com/watch?v=WQMDZniQgj4

# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [4]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):

  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics # could see tuning this down to 40 but can't tell if it would pick up more or less cats
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  # print('    passed cat test:', would_pass)
  return would_pass

In [5]:
# on colab you can set these userdata properties by clicking the key on the left bar, creating a secret, and giving the colab "notebook access"
BSKY_USERNAME = userdata.get('bsky_username')
BSKY_PASSWORD = userdata.get('bsky_password')

client = Client()
client.login(BSKY_USERNAME, BSKY_PASSWORD)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.com', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Self-taught professional software developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2_100%, ex streamer, ex melee player. \nGrilling, coding, gaming, magic:tg, tcgs, and most importantly - posting pictures of my cat. 😻\nNO POLITICS\nhe/him', display_name='Tyro, ft Ricky', followers_count=1211, follows_count=2515, indexed_at='2025-01-21T01:55:44.913Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(c

In [6]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ ✅ Successfully liked post and followed user: {post.author.handle}')
  return followed_user
  # TODO - add this friend and the followed-user uri to the database

In [7]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'
EMBEDDED_VID = 'app.bsky.embed.video#view'
FEED_CATURDAY = 'at://did:plc:pmyqirafcp3jqdhrl7crpq7t/app.bsky.feed.generator/aaad4sb7tyvjw'
FEED_SIAMESE = 'at://did:plc:jv3qdc5vxujp6taaa7nte35i/app.bsky.feed.generator/aaac6wmikqyhq'
FEED_CATPICS = 'at://did:plc:q6gjnaw2blty4crticxkmujt/app.bsky.feed.generator/cv:cat'
FEED_CATS = 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats'
FEED_NAME = {FEED_CATURDAY: "'Caturday'", FEED_SIAMESE: "'Siamese Cats'", FEED_CATPICS: "'Cat Pics'", FEED_CATS: "'Cats!'"}
URL_BEGIN = 'https://bsky.app/profile/'
URL_POST = '/post/'

# query creation constants
SCHEMA = 'bsky'
TABLE_FOLLOWS = 'follows'
COL_USERID = 'user_id'
COL_USERHANDLE = 'user_handle'
COL_FOLLOWURI = 'follow_uri'
COL_FOLLOWDATE = 'follow_date'
COL_FOLLOWSYOU = 'follows_you'
DATETIME_NOW = 'NOW()'
END_LINE = ','
END_QUERY = ';'


def createPostUrl(feed_post):
  url_handle = feed_post.post.author.handle
  url_ending_index = feed_post.post.uri.find('.feed.post/') + 11
  url_ending = feed_post.post.uri[url_ending_index : ]
  return URL_BEGIN + url_handle + URL_POST + url_ending

# TODO - The way I have this written the function probably breaks if we reach the end of the feed, so I need to write some kind of case to handle that. The feeds I'm using there shouldn't be an end to reach.
# But technically if you set the number of posts to check to like a billion and number of new followers to follow to like a billion it could reach the end and break.
# so please use responsibly

def follow_more_users(post_count, follows_count, feed):
  posts_to_check = post_count
  next_page = ''
  new_follow_count = 0
  page_count = 0
  query_created_of_users_added = []
  query_created_of_users_added.append(f'INSERT INTO "{SCHEMA}"."{TABLE_FOLLOWS}" ({COL_USERID}, {COL_USERHANDLE}, {COL_FOLLOWURI}, {COL_FOLLOWDATE}, {COL_FOLLOWSYOU}) ')
  query_created_of_users_added.append(f'VALUES ')
  already_added_dids = set()

  while posts_to_check > 0:
    print(f'[checking page {page_count} of feed {FEED_NAME[feed]}, {posts_to_check} posts left to check, and have found {new_follow_count} new users to follow]')
    page_count += 1
    limit = min(posts_to_check, 100)
    posts_to_check -= limit
    data = client.app.bsky.feed.get_feed({
        'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
        'limit': limit,
        'cursor': next_page
    }, headers={})
    next_page = data.cursor

    for i, f in enumerate(data.feed):
      you_follow_them = f.post.author.viewer.following
      you_are_followed_by = f.post.author.viewer.followed_by
      did = f.post.author.did

      if False: # TODO IF POST ALREADY SEEN
        print(f'{i} - 👀 skipped. Post {""} has already been viewed.') # add post id here idk
        continue
      else:
        # TODO - add post did to set to mark as seen
        if you_follow_them and you_are_followed_by:
          print(f'{i} 💕 user: {f.post.author.handle} is a mutual follower. Liking this post. {createPostUrl(f)}')
          liked_post = client.like(uri=f.post.uri, cid=f.post.cid).uri
        elif did in already_added_dids:
          print(f'{i} ✗ 👀 user: {f.post.author.handle} was already followed in this batch.')
        elif you_follow_them or you_are_followed_by:
          print(f'{i} ✗ 👀 user: {f.post.author.handle} {"already follows you." if you_are_followed_by else ""}{"is already being followed." if you_follow_them else ""}')
        ## TODO - add elif check here for if they are in the database as someone who was once our friend? not necessary, could just immediately cycle them back into the friend rotation, but do this if you don't want to readd people
        elif not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
          if f.post.embed.py_type == EMBEDDED_VID:
            print(f'{i} ✗ 🎥 video post: {createPostUrl(f)}')
          else:
            print(f'{i} ✗ 🔲 no pic for post {i}')
        else:
          print(i, '✓', '📷', f.post.embed.images[0].fullsize)
          print(f'    post: {createPostUrl(f)}')
          is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
          if is_cat:
            print(f'    ✓✓ 😺 successfully found cat pic at post {i}.')
            new_follow_count += 1
            handle = f.post.author.handle
            follow_uri = like_post_and_add_user(f.post)
            date_added = DATETIME_NOW
            follows_you = False
            if len(query_created_of_users_added) > 2:
              # first one don't add a comma but add one after every one after that
              query_created_of_users_added.append(END_LINE)
            query_created_of_users_added.append(f"    ('{did}', '{handle}', '{follow_uri}', {date_added}, '{follows_you}') ")
            already_added_dids.add(did)
            if new_follow_count == follows_count:
              print(f'Successfully followed {new_follow_count} new users!')
              query_created_of_users_added.append(f'RETURNING {COL_USERHANDLE}, {COL_FOLLOWDATE}{END_QUERY}')
              return query_created_of_users_added
          else:
            print(f'    ✓✗ ❌ post {i} was not a cat pic')

  print(f'ran out of posts to check for new followers. followed {new_follow_count} new users.')
  query_created_of_users_added.append(f'RETURNING {COL_USERHANDLE}, {COL_FOLLOWDATE}{END_QUERY}')
  return query_created_of_users_added

    # print(data.feed[0].post.embed.images[0].fullsize)

In [8]:
CATURDAY_DOW = 'Saturday'
USER_TIMEZONE = "US/Eastern" # you should fill this in with your own timezone here
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))

dow = cur_timestamp.strftime("%A")
is_caturday = dow == CATURDAY_DOW
query_caturday = []

if is_caturday:
  print("IT'S CATURDAY! Checking the Caturday feed for new followers.")
  query_caturday = follow_more_users(500, 100, FEED_CATURDAY)
else:
  print("Sorry, today's not Caturday.")

Sorry, today's not Caturday.


In [9]:
for line in query_caturday:
  print(line)

In [10]:
query_siamese = follow_more_users(300, 200, FEED_SIAMESE)

[checking page 0 of feed 'Siamese Cats', 300 posts left to check, and have found 0 new users to follow]
0 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:xseasd45spwvtll5wqlmeqx4/bafkreifyrcqvzelxcdr6ebqcknoqzt6zgrggi6rdergivm25yus34lcse4@jpeg
    post: https://bsky.app/profile/rhymeswithspecimen.bsky.social/post/3lgewvwnpcs2t


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'return_tensor'
  return self.preprocess(images, **kwargs)


    found cat label: 3
    found bad label: -1 []
    ✓✓ 😺 successfully found cat pic at post 0.
      ✓✓✓ ✅ Successfully liked post and followed user: rhymeswithspecimen.bsky.social
1 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:unnk762zecicobq6ukxa4fgn/bafkreidfj77ywjti2u4ay256fzyfoloutch72xpdpwffkw7wupg6yatohi@jpeg
    post: https://bsky.app/profile/sunpixie.bsky.social/post/3lgewvujhps2y
    found cat label: 0
    found bad label: -1 []
    ✓✓ 😺 successfully found cat pic at post 1.
      ✓✓✓ ✅ Successfully liked post and followed user: sunpixie.bsky.social
2 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:j4wyouwmibpqxo6cpdamf5cs/bafkreieoaiu4jlqd2bhu3jie3iol7ric5fy26377hinwz2xlbrhzqrtcim@jpeg
    post: https://bsky.app/profile/stefistarlite.bsky.social/post/3lgewvr7cb22m
    found cat label: 3
    found bad label: 48 [916, 'web site, website, internet site, site']
    ✓✗ ❌ post 2 was not a cat pic
3 ✗ 👀 user: baxtercat.bsky.social is already being followed.
4

In [11]:
for line in query_siamese:
  print(line)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:xseasd45spwvtll5wqlmeqx4', 'rhymeswithspecimen.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexob64342p', NOW(), 'False') 
,
    ('did:plc:unnk762zecicobq6ukxa4fgn', 'sunpixie.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexoct3la2f', NOW(), 'False') 
,
    ('did:plc:lquq5f7qp4dcqdnn55vci5xs', 'meyowmix.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexog5qrn2c', NOW(), 'False') 
,
    ('did:plc:edhdpmfgwtbgtlb5f272suso', 'bluebird1988.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexoicotf2c', NOW(), 'False') 
,
    ('did:plc:okxap7sueqcjpu7p5gqn7k2y', 'lizethducks.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexolgepo24', NOW(), 'False') 
,
    ('did:plc:inizqpfstzq2m6xidktumdiq', 'gooddeeds-dee.bsky.social', 'at://did

In [12]:
query_cats = follow_more_users(200, 100, FEED_CATS)

[checking page 0 of feed 'Cats!', 200 posts left to check, and have found 0 new users to follow]
0 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:eynnkecvdzfozekfyksjyed2/bafkreibxm754nkepqbyek3cot6pzlxazrocoz747ebncks5y4jy45qsihi@jpeg
    post: https://bsky.app/profile/stephwrites.bsky.social/post/3lgexiotn5s2s
    found cat label: 2
    found bad label: -1 []
    ✓✓ 😺 successfully found cat pic at post 0.
      ✓✓✓ ✅ Successfully liked post and followed user: stephwrites.bsky.social
1 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:vpuozplkqp7hs7a2xs4pbgy6/bafkreic67zpoejqnhr276frx5zletdne424wnwpipqqxwjjcbwkrb2nyd4@jpeg
    post: https://bsky.app/profile/compukitty.bsky.social/post/3lgexia3ql22a
    found cat label: 0
    found bad label: -1 []
    ✓✓ 😺 successfully found cat pic at post 1.
      ✓✓✓ ✅ Successfully liked post and followed user: compukitty.bsky.social
2 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:6l4vxr4bymyz4yugfyomadgu/bafkreifbtq5rtl

In [13]:
# set_lines = set()
# for line in query_cats[2:-1]:
#   # print(line)
#   set_lines.add(line)
# # print(set_lines)
# print(query_cats[0])
# print(query_cats[1])
# set_lines.remove(',')
# for i, line in enumerate(set_lines):
#   if i != 0:
#     print(',')
#   print(line)
# print(query_cats[-1])

for line in query_cats:
  print(line)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:eynnkecvdzfozekfyksjyed2', 'stephwrites.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexxvpw6i2f', NOW(), 'False') 
,
    ('did:plc:vpuozplkqp7hs7a2xs4pbgy6', 'compukitty.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexxx2w7g2x', NOW(), 'False') 
,
    ('did:plc:6l4vxr4bymyz4yugfyomadgu', 'christinaheacock.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexxyloox22', NOW(), 'False') 
,
    ('did:plc:r7gjoarajtin2p6dmgy2ahdh', 'ricocat.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexxzsfhv23', NOW(), 'False') 
,
    ('did:plc:7wulfbf3dnpyypa6ozef63j4', 'thentwo.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgexy54o5u2p', NOW(), 'False') 
,
    ('did:plc:hyvbm4u642mv5ulrjq74ruuk', 'nick.mudry.me', 'at://did:plc:ktkc7jfakxzjp

In [ ]:
# cat pics is the smallest hashtag, and is the least important to check by far, usually will only get like 25 of the 100
query_catpics = follow_more_users(100, 100, FEED_CATPICS)

[checking page 0 of feed 'Cat Pics', 100 posts left to check, and have found 0 new users to follow]
0 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:soedgzlqyvy2rxvueqsnfij6/bafkreicx6rzvsfzgwlsk7dc7rhxjahjccnlh2nudk2gwzcceoh3l5xuoze@jpeg
    post: https://bsky.app/profile/juanjo-el-guapo.bsky.social/post/3lgbofop6fk2k
    found cat label: -1
    found bad label: 13 [916, 'web site, website, internet site, site']
    ✓✗ ❌ post 0 was not a cat pic
1 ✗ 👀 user: chiyopurrs.bsky.social is already being followed.
2 ✗ 🎥 video post: https://bsky.app/profile/pj-history.bsky.social/post/3lgbof4e3c22c
3 ✗ 🎥 video post: https://bsky.app/profile/widowbee.bsky.social/post/3lgboeqnot22d
4 ✓ 📷 https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:d6lckz7hiecf3j5uidz2l5c2/bafkreihqnyy7sskvbn6o6f4fv3u7fryfl3ekjwxiainh4ft5dbiqjsw5uy@jpeg
    post: https://bsky.app/profile/kitchenjedi.bsky.social/post/3lgboenqob22r
    found cat label: -1
    found bad label: 17 [917, 'comic book', 921, 'book jac

In [ ]:
for line in query_catpics:
  print(line)

INSERT INTO "bsky"."follows" (user_id, user_handle, follow_uri, follow_date, follows_you) 
VALUES 
    ('did:plc:fiazz7xhqrlljapay3243aws', 'luciafasano.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgboxgzwc42c', NOW(), 'False') 
,
    ('did:plc:nqlj4cxxw7657anuwqz3wjwf', 'promocorner.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgboxhyywi2x', NOW(), 'False') 
,
    ('did:plc:mia5sdanhkhtuavz7ussyz5p', 'gigiomics.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgboxj27vu2c', NOW(), 'False') 
,
    ('did:plc:6wfileqf4n4u5bamf6xvt7vv', 'elitistscrub.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgboxk6l262l', NOW(), 'False') 
,
    ('did:plc:fpvzqwnkq65qo6es3q4pglxi', 'jennishutterfly.bsky.social', 'at://did:plc:ktkc7jfakxzjpooj52ffc6ra/app.bsky.graph.follow/3lgboxlpcey2f', NOW(), 'False') 
,
    ('did:plc:gvjynel5sru67tswewkktyje', 'fiercethedrittten.bsky.social', 'at: